## LSTM 

### a)In this problem, we are trying to build a generative model to mimic the writing style of prominent British Mathematician, Philosopher, prolific writer, and political activist, Bertrand Russell.

In [2]:
import pandas as pd
import numpy as np

### b) Download the following books from Project Gutenberg http://www.gutenberg.org/ebooks/author/355 in text format: <br>
i. The Problems of Philosophy<br>
ii. The Analysis of Mind<br>
iii. Mysticism and Logic and Other Essays<br>
iv. Our Knowledge of the External World as a Field for Scientific Method in Philosophy

In [3]:
filename="book1.txt"
raw_text=open(filename,encoding='utf-8').read()
raw_text=raw_text.lower()

In [4]:
#print(raw_text)

### c)ii.Use a character-level representation for this model by using extended ASCII that has N = 256 characters. Each character will be encoded into a an integer using its ASCII code. Rescale the integers to the range [0,1]

In [5]:
chars=sorted(list(set(raw_text)))
char_to_int=dict((c, i) for i, c in enumerate(chars))

In [6]:
print(char_to_int)

{'\n': 0, ' ': 1, '!': 2, '"': 3, '&': 4, "'": 5, '(': 6, ')': 7, '*': 8, '+': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '=': 26, '>': 27, '?': 28, '[': 29, ']': 30, '_': 31, 'a': 32, 'b': 33, 'c': 34, 'd': 35, 'e': 36, 'f': 37, 'g': 38, 'h': 39, 'i': 40, 'j': 41, 'k': 42, 'l': 43, 'm': 44, 'n': 45, 'o': 46, 'p': 47, 'q': 48, 'r': 49, 's': 50, 't': 51, 'u': 52, 'v': 53, 'w': 54, 'x': 55, 'y': 56, 'z': 57, '{': 58, '|': 59, '}': 60, '\xa0': 61, '§': 62, '·': 63, 'â': 64, 'æ': 65, 'è': 66, 'é': 67, 'ë': 68, 'î': 69, 'ï': 70, 'ô': 71, 'ö': 72, 'ü': 73, 'œ': 74, 'ŭ': 75, 'α': 76, 'β': 77, 'γ': 78, 'η': 79, 'θ': 80, 'ι': 81, 'κ': 82, 'λ': 83, 'ν': 84, 'ο': 85, 'π': 86, 'ρ': 87, 'σ': 88, 'τ': 89, 'φ': 90, 'ὴ': 91, 'ή': 92, 'ί': 93, 'ὸ': 94, 'ό': 95, '′': 96, '″': 97, '\ufeff': 98}


In [7]:
n_chars=len(raw_text)
n_vocab=len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  1577211
Total Vocab:  99


### c)iii. iv. Choose a window size W = 100. <br> Inputs to the network will be the first W-1 = 99 characters of each sequence,and the output of the network will be the Wth character of the sequence.

In [8]:
window=100 #Window size=100
dataX = []
dataY = []
for i in range(0, n_chars-window, 1):
	seq_in = raw_text[i:i + window]
	seq_out = raw_text[i + window]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  1577111


In [48]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

### c) v. Encode the output using one-hot encoding scheme

In [12]:
X = np.reshape(dataX, (n_patterns, window, 1))

X = X / float(n_vocab)

y = np_utils.to_categorical(dataY)

In [13]:
y.shape

(1577111, 98)

### c) vi.,vii.,viii.

In [14]:
lstm=Sequential()
lstm.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
lstm.add(Dropout(0.2))
lstm.add(Dense(y.shape[1], activation='softmax'))
lstm.compile(loss='categorical_crossentropy', optimizer='adam')

In [15]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

### c) ix. Choose a reasonable number of epochs for training, considering your computational power. 
## Using 20 epochs

In [16]:
lstm.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
1577111/1577111 [==============================] - 4195s 3ms/step - loss: 2.7420

Epoch 00001: loss improved from inf to 2.74202, saving model to weights-improvement-01-2.7420.hdf5
Epoch 2/20
1577111/1577111 [==============================] - 4244s 3ms/step - loss: 2.4698

Epoch 00002: loss improved from 2.74202 to 2.46980, saving model to weights-improvement-02-2.4698.hdf5
Epoch 3/20
1577111/1577111 [==============================] - 4257s 3ms/step - loss: 2.2757

Epoch 00003: loss improved from 2.46980 to 2.27567, saving model to weights-improvement-03-2.2757.hdf5
Epoch 4/20
1577111/1577111 [==============================] - 4207s 3ms/step - loss: 2.1552

Epoch 00004: loss improved from 2.27567 to 2.15515, saving model to weights-improvement-04-2.1552.hdf5
Epoch 5/20
1577111/1577111 [==============================] - 4187s 3ms/step - loss: 2.0744

Epoch 00005: loss improved from 2.15515 to 2.07437, saving model to weights-improvement-05-2.0744.hdf5
Epoch 6/20
1577111/15771

### c) x.Use model checkpointing to keep the network weights to determine each time an improvement in loss is observed at the end of the epoch. Find the best set of weights in terms of loss.

In [19]:
filename="weights-improvement-20-1.7438.hdf5"
lstm.load_weights(filename)
lstm.compile(loss='categorical_crossentropy', optimizer='adam')

### c) xi.Use the network with the best weights to generate 1000 characters, using the following text as initialization of the network: 
### There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object.

In [20]:
int_to_char=dict((i, c) for i, c in enumerate(chars))

In [49]:
#int_to_char

In [41]:
seed_sentence = [char_to_int[char] for char in "there are those who take mental phenomena naively, just as they would physical phenomena. this school of psychologists tends not to emphasize the object."]

In [42]:
import sys

In [47]:
pattern = seed_sentence[0:100]

for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    
    prediction=lstm.predict(x, verbose=0)
    index=np.argmax(prediction)
    result=int_to_char[index]
    seq_in=[int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern=pattern[1:len(pattern)]

ls of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of the semses of 